# Week1

Notes from week1 of algorithms course


## Steps to develop usable algorithm

* model the problem
* find an algorithm that solves it
* is it fast enough? is it memory efficient enough?
* if not, think about why
* find a way to address the shortcomings
* iterate until satisfied

## Dyanmic connectivity
* Given set of N objects
    * union command: connect two objects
    * find/connected query: is there a path connecting the two objects
*     

In [46]:
class Quick_find:
    def __init__(self, n):
        self._ids = [i for i in range(n)]

    def are_connected(self, first: int, second: int) -> bool:
        '''
        O(1) time and space
        '''
        try:
            return self._ids[first] == self._ids[second]
        except IndexError as e:
            print('invalid id provided ')
            print(e)

    def do_union(self, first: int, second: int) -> None:
        '''
        go through each element in the array and make sure we identify which
        node is in the same collection as first (_ids[first] has the same value as the node in question) 
        and we connect all these nodes to _ids[second] (replace this value with whatever second has)
        At most 2n + 2 accesses
        O(n) time and O(1) space 
        '''
        ids = self._ids
        first_val = ids[first]
        second_val = ids[second]
        for i in range(len(ids)):
            if ids[i] == ids[first]: # connected to first
                ids[i] = ids[second]
        print(self._ids)
        
            

In [47]:
q = Quick_find(10)


In [48]:
q.are_connected(10,30)

invalid id provided 
list index out of range


In [49]:
q.do_union(0,4)
q.do_union(5,6)
q.do_union(6,9)

assert q.are_connected(5,9), "5 and 9 should be connected"
try:
    assert q.are_connected(0,9)
except AssertionError:
    print('expected that 0, 9 are not connected')
else:
    raise Exception('should not be connected')


[4, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[4, 1, 2, 3, 4, 6, 6, 7, 8, 9]
[4, 1, 2, 3, 4, 9, 9, 7, 8, 9]
expected that 0, 9 are not connected


* Quick Find is too slow
* if you do N union operations then we end up with N^2 operations
* Since 1950s it takes about 1 second to access all words in main memory
    * 10^9 operations a second, with 10^9 words of main memory => 1 second
* with quick find since it is N^2, it will take 10^18 operations to do 10^9 union commands on 10^9 objects
    * this is 30 years of compute time!

**Lesson** ==> quadratic algorithms don't scale with technology!


Union find involves modeling the collection of connected nodes as part of a tree

As we do union we are adding the first node's subtree as a child of the second node's subtree

In [52]:
class Union_Find:
    def __init__(self, n):
        # initially each node is a parent of itself
        self._ids = [i for i in range(n)]
        
    def _get_root(self, idx: int) -> int:
        '''
        private helper to get the root node of a given node
        '''
        while self._ids[idx] != idx:
            idx = self._ids[idx]
        return idx
        
    def connected(self, first: int, second: int) -> bool:
        return self._get_root(first) == self._get_root(second)
        
    def union(self, first, second):
        first_root = self._get_root(first)
        second_root = self._get_root(second)
        self._ids[first_root] = second_root


In [53]:
uf = Union_Find(10)
assert uf.connected(9,9), 'node should be connected to itself'
uf.union(0,8)
uf.union(3,5)
uf.union(6,8)

assert uf.connected(0,8)



* quick union is too slow too since the find and union are both O(n) time
* trees can grow too tall

  A solution is to used weighted union find. Keep track of height/size of trees and always put smaller trees under taller trees to keep the average height lower
  One key point not mentioned in lecture is that the constructor sets up the sizes to be 1 by default for each node.
  

In [69]:
class Union_find_weighted:
    def __init__(self, n):
        # initially each node is a parent of itself
        self._ids = [i for i in range(n)]
        self._sizes = [1 for _ in range(n)]
        
    def _get_root(self, idx: int) -> int:
        '''
        private helper to get the root node of a given node
        '''
        while self._ids[idx] != idx:
            idx = self._ids[idx]
        return idx
        
    def are_connected(self, first: int, second: int) -> bool:
        return self._get_root(first) == self._get_root(second)
        
    def do_union(self, first, second):
        first_root = self._get_root(first)
        second_root = self._get_root(second)
        if first_root == second_root:
            return
        sz = self._sizes
        ids = self._ids
        # put shorter tree under the taller tree
        if sz[first_root] < sz[second_root]:
            ids[first_root] = sz[second_root]
            sz[second_root] += sz[first_root]
        else:
            ids[second_root] = first_root
            sz[first_root] += sz[second_root]

    def __str__(self):
        return f"nodes :{self._ids}, sizes:{self._sizes}"
        

In [70]:
uf = Union_find_weighted(10)
uf.do_union(2,4)
uf.do_union(6,9)
uf.do_union(2,6)
uf.do_union(1,9)

print(uf)



nodes :[0, 4, 2, 3, 2, 5, 2, 7, 8, 6], sizes:[1, 1, 5, 1, 1, 1, 2, 1, 1, 1]


* Running time is proportional to depth of first and second
* union takes O(1) time given roots
* depth of any node is at most lg(n) --> log base-2
    * for n=1000 depth is 3, for n=10^9, the depth is 30! 


> TODO: add details about why this is true, the math proof

> TODO: add a print tree function